In [2]:
#Applying Bert to  Stanford Question Answering Dataset

In [48]:
!pip install transformers
!pip install PyMuPDF
!pip install langdetect
!pip install rake-nltk
!pip install sentence-transformers
!pip install langchain
!pip install PyPDF2

In [4]:
import torch

In [5]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from transformers import BertTokenizer
tokenizer= BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # Tokenize
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # Set Segment-IDs
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Run example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [8]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80)

bert_abstract = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."

print(wrapper.fill(bert_abstract))

We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language
representation models (Peters et al., 2018a; Radford et al., 2018), BERT is
designed to pretrain deep bidirectional representations from unlabeled text by
jointly conditioning on both left and right context in all layers. As a result,
the pre-trained BERT model can be finetuned with just one additional output
layer to create state-of-the-art models for a wide range of tasks, such as
question answering and language inference, without substantial taskspecific
architecture modifications. BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art results on eleven natural language
processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute
improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1
question answering Test F1 to 93.2 (1.5 point absolute improvement) 

In [9]:
question = "What does the BERT stand for?"

answer_question(question, bert_abstract)

Query has 258 tokens.

Answer: "bidirectional encoder representations from transformers"


In [38]:
import logging as logger
import math
import os
import sqlite3

import fitz
import matplotlib.pyplot as plt
import numpy as np
from langdetect import detect
from rake_nltk import Rake
from scipy.signal import argrelextrema
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import nltk
nltk.download(['stopwords', 'punkt'])
from transformers import AutoTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')


logger.basicConfig(level=logger.INFO)
# TODO: implement function to get matched documents based in user prompt
# TODO: add logging to file
# TODO: check for syntax and formatting
# TODO: add comments
# TODO: implement method to preprocess also .txt files (currently only pdf, compare line 35)
# TODO: fix the bug that the filename is currently not written to db (compare line 288)


class FileProcessor:
    def __init__(self, file):
        self.tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        logger.info("Initializing FileProcessor")
        self.document = None
        # read in pdf document
        try:
            file_content = file.getvalue()
            self.document = fitz.open("pdf", file_content)
            logger.info("PDF document loaded")
        except Exception as e:
            logger.error(f"Could not open PDF file: {e}")
        self.document_content = {"text": str(), "images": dict()}

    @staticmethod
    def extract_text(
        document: fitz.fitz.Document, extract_text_from_image: bool = False
    ) -> str:
        """extract text from a pdf page
        Args:
            page (fitz.fitz.Page): page from pdf document
            extract_from_image (bool, optional): extract text from image using OCR (not possible yet). Defaults to False.
        Returns:
            str: text from page
        """

        if extract_text_from_image:
            logger.info("Extracting text from image")
            pass

        else:
            logger.info("Extracting text from pages")
            text = str()
            for page in document:
                try:
                    text += page.get_text() + " "
                except Exception as e:
                    print(f"Error processing file on page {page.number}: {e}")
                    continue
            return text
    @staticmethod
    def calc_rev_sigmoid(x: float) -> float:
        """calculate reverse sigmoid function

        Args:
            x (float): input value

        Returns:
            float: output value
        """
        return 1 / (1 + math.exp(0.5 * x))

    @staticmethod
    def activate_similarities(similarities: np.array, p_size=10) -> np.array:
        """calculate activated similarities using reverse sigmoid function

        Args:
            similarities (np.array): similarities between sentences
            p_size (int, optional): size of sigmoid function. Defaults to 10.

        Returns:
            np.array: activated similarities
        """
        x = np.linspace(-10, 10, p_size)
        y = np.vectorize(FileProcessor.calc_rev_sigmoid)
        activation_weights = np.pad(y(x), (0, similarities.shape[0] - p_size))
        diagonals = [
            similarities.diagonal(each) for each in range(0, similarities.shape[0])
        ]
        diagonals = [
            np.pad(each, (0, similarities.shape[0] - len(each))) for each in diagonals
        ]
        diagonals = np.stack(diagonals)
        diagonals = diagonals * activation_weights.reshape(-1, 1)
        activated_similarities = np.sum(diagonals, axis=0)
        return activated_similarities


    def split_text_into_chunks(
      self,
      text: str,
      filename: str,
      visualize_splitting: bool = False) -> list:
      doc_lang = detect(text)
      stop_words = set(stopwords.words('german')) if doc_lang == "de" else set(stopwords.words('english'))

      rake = Rake(stopwords=stop_words)

      logger.info("Loading model")
      model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
      sentences = text.split(". ")

      logger.info("Encoding sentences")
      embeddings = model.encode(sentences)
      similarities = cosine_similarity(embeddings, embeddings)
      logger.info("Calculating activated similarities")
      activated_similarities = self.activate_similarities(similarities, p_size=5)
      minima = argrelextrema(activated_similarities, np.less, order=2)
      split_points = [each for each in minima[0]]

      if visualize_splitting:
          self.plot_chunk_points(activated_similarities, split_points)

      logger.info("Creating chunks list")
      chunks = []
      text_chunk = str()
      for split_point, sentence in enumerate(sentences):
          text_chunk += sentence + ". "
          if split_point in split_points:
              rake.extract_keywords_from_text(text_chunk)
              extracted_keywords = rake.get_ranked_phrases()[:5]
              chunks.append(text_chunk)
              text_chunk = str()
      if text_chunk != str():
          rake.extract_keywords_from_text(text_chunk)
          chunks.append(text_chunk)

      final_chunks = []
      for chunk in chunks:
            if len(self.tokenizer.tokenize(chunk)) > 512:
                final_chunks.extend(self.divide_and_conquer(chunk))
            else:
                final_chunks.append(chunk)

      return final_chunks


    def divide_and_conquer(self, chunk: str) -> list:
      sentences = nltk.sent_tokenize(chunk)
      new_chunk = ""
      sub_chunks = []

      for sentence in sentences:
          # Wenn der aktuelle Satz zu lang ist, aufteilen
          if len(self.tokenizer.tokenize(sentence)) > 512:
              # Teilen des langen Satz in kleinere Teile
              sub_chunks.extend(self.divide_long_sentence(sentence))
          elif len(self.tokenizer.tokenize(new_chunk + sentence)) > 512:
              sub_chunks.append(new_chunk.strip())
              new_chunk = sentence
          else:
              new_chunk += " " + sentence

      if new_chunk:
          sub_chunks.append(new_chunk.strip())

      return sub_chunks

    def divide_long_sentence(self, sentence: str) -> list:
      words = sentence.split()
      sub_sentences = []
      current_chunk = []

      for word in words:
          current_chunk.append(word)
          # Prüfen, ob die Länge des aktuellen Chunks die Grenze erreicht hat
          if len(self.tokenizer.tokenize(' '.join(current_chunk) + ' ' + word)) > 512:
              sub_sentences.append(' '.join(current_chunk))
              current_chunk = []

      # Fügen letzten Chunk hinzu, falls vorhanden
      if current_chunk:
          sub_sentences.append(' '.join(current_chunk))

      return sub_sentences



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
# PDF-Dokument öffnen
with fitz.open('Databases - 10-IndexingHashing.pdf') as doc:
  processor = FileProcessor(doc)
  text = processor.extract_text(doc)

ERROR:root:Could not open PDF file: 'Document' object has no attribute 'getvalue'


In [40]:
filename = 'Databases - 10-IndexingHashing.pdf'
chunks= processor.split_text_into_chunks(text, filename)

Token indices sequence length is longer than the specified maximum sequence length for this model (3865 > 512). Running this sequence through the model will result in indexing errors


In [49]:
chunks[5]

'A secondary index which is created on a key (unique) ﬁeld\n(secondary key) has one index entry for each record\n–\xa0represents dense index and has pointer to the block of the record or\nto the record itself\n–\xa0this corresponds to any UNIQUE key attribute\n2. A secondary index which is created on a non-key ﬁeld can have for\neach index entry (indexing ﬁeld) more corresponding records in the\ndata ﬁle\n–\xa0pointer of the index entry has address of the location of the block\nwith record pointers to the actual records in the data ﬁle\n–\xa0represents sparse secondary index\n17\n Secondary index on a key ﬁeld\nSecondary index on a key ﬁeld\nDense secondary index on a secondary key attribute\n18\n Secondary index on a non-key ﬁeld\nSecondary index on a non-key ﬁeld\nSparse secondary index on a non-key attribute\n19\n Searching for a record in ﬁle - example\nSearching for a record in ﬁle - example\nSuppose that we have a ﬁle with records of ﬁxed size:\n–\xa0number of records \n–\xa0reco

In [52]:
question = "What kind of secondary indexes exist?"

answer_question(question, chunks[5])

Query has 484 tokens.

Answer: "dense"
